Загрузим необходимые библиотеки

In [ ]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

In [ ]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

#normalizing data
from sklearn.preprocessing import StandardScaler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

Проверка работоспособности и качества пайплайна

Загрузим данные

In [ ]:
X_test = pd.read_csv("X_test_cardio.csv")
y_test = pd.read_csv("y_test_cardio.csv")

In [ ]:
X_test.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
0,66728,21770,1,156,64.0,140,80,2,1,0,0,1
1,69098,21876,1,170,85.0,160,90,1,1,0,0,1
2,59185,23270,1,151,90.0,130,80,1,1,0,0,1


In [ ]:
y_test.head(3)

,cardio
0,1
1,1
2,1


Загрузим пайплайн

In [ ]:
with open('cardio_pipeline_lr.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [ ]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='age'))])),
                                                ('gender',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='gender'))])),
                                                ('height',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='height'))])),
                                                ('weight',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='weight'))])),
                             

Проверим его работоспособность

In [ ]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None)

Посмотрим первые 10 значений вероятностей

In [ ]:
preds[:10]

array([0.70287408, 0.78248027, 0.81694631, 0.638395  , 0.39559811,
       0.28697793, 0.30784091, 0.36854435, 0.79033133, 0.80694086])

И посчитаем метрики:

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.39919894878919676, F-Score=0.719, Precision=0.623, Recall=0.851


Убедились, что модель работает